# B3 — Seleção de Fechamentos de Loop (Difícil)

Selecione um **conjunto independente de peso máximo** de candidatos a fechamento de loop (grafo de conflitos). Incentiva uso de QAOA/Ising; baseline guloso.

### Contexto
Em SLAM baseado em grafos, **candidatos a fechamento de loop** são correspondências entre o keyframe atual e poses passadas. Cada candidato adiciona uma restrição (aresta), porém hipóteses conflitantes (aliasing perceptual ou matches mutuamente inconsistentes) não podem ser aceitas juntas. Conflitos são modelados como arestas: nós = hipóteses de loop-closure, arestas = conflitos. Selecionar um subconjunto consistente vira um problema de **Maximum-Weight Independent Set (MWIS)** onde pesos codificam confiança geométrica ou ganho de informação. [`common.data_utils.mwis_loop_closure_instance`](common/data_utils.py) gera grafos de conflito e pesos de confiança.

### Por que quântico?
MWIS admite formulação Ising/QUBO com variáveis binárias $z_i\in\{0,1\}$ impondo penalidades de independência. Instâncias difíceis são combinatórias e altamente não convexas — terreno propício para QAOA ou heurísticas de annealing quântico. Amostragem quântica combinada a pós-processamento clássico pode diversificar subconjuntos de alto valor além do heurístico guloso.

## Especificação de Entrada e Saída
**Entrada:** `G (networkx.Graph)`, `w (np.ndarray)` pesos por nó  
**Saída:** `S (list[int])` índices do conjunto independente  
Orientações de recursos/runtime: qubits ≤ 12, passos do otimizador ≤ 250; manter tempo total ≤ ~10 s.

### Dicas de solução
- Monte o Hamiltoniano Ising de MWIS $H = -\sum_i w_i z_i + \lambda \sum_{(i,j)\in E} z_i z_j$ com penalidade $\lambda > \max_i w_i$.
- Use [`common.quantum_utils.default_device`](common/quantum_utils.py) com camadas customizadas ou QAOA para amostrar bitstrings; amostragem (shots) aproxima probabilidades.
- Repare bitstrings amostrados em conjuntos independentes viáveis (removendo conflitos pelos menores pesos marginais) e retenha o melhor valor.
- Truque híbrido: warm-start do QAOA com heurística gulosa ou refine a solução reparada com busca local clássica (trocas que melhoram peso).

## Setup

In [ ]:
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
np.random.seed(1337)


## Baseline (referência)

In [ ]:
import networkx as nx
G, w = du.mwis_loop_closure_instance(n=40, p_edge=0.12, seed=0)
S = bl.baseline_mwis(G, w)
val = w[S].sum()
print("Valor MWIS baseline:", float(val))


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(G, w):
    # Esboço:
    # 1. Codificar (G, w) em Ising / QUBO (variáveis z_i ∈ {0,1}).
    # 2. Rodar QAOA ou sampler quântico (ver common.quantum_utils).
    # 3. Reparar bitstrings em conjuntos independentes viáveis.
    # 4. (Opcional) Refino clássico local; retornar índices ordenados.
    return bl.baseline_mwis(G, w)


## Testes públicos (rápidos)

In [ ]:
G, w = du.mwis_loop_closure_instance(n=50, p_edge=0.10, seed=3)
S = solve(G, w)
# Verificação de viabilidade
for u, v in G.edges():
    assert not (u in S and v in S), "Não é independente"
val = w[S].sum()
val_base = w[bl.baseline_mwis(G, w)].sum()
print("Seu valor:", float(val), "Baseline:", float(val_base))
assert val >= 0.92 * val_base - 1e-6, "Muito abaixo do baseline"
print("OK")


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.